# Merge Dữ Liệu KCS và LF-Log

Notebook này xử lý và merge dữ liệu từ 2 nguồn:
- **KCS (20260129.csv)**: Dữ liệu lấy mẫu tháng 10, 11, 12 với ID mẻ dạng `25B006105` (25 = năm 2025, B = lò B, 006105 = số mẻ)
- **LF-Log.csv**: Dữ liệu nhật ký vận hành tháng 12 với ID mẻ dạng `C7536` (C = lò thổi C, 7536 = số mẻ)

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## 1. Load Dữ Liệu

In [2]:
# Load file KCS
kcs_df = pd.read_csv('01-data/KCS/20260129.csv')
print(f"KCS data shape: {kcs_df.shape}")
print(f"KCS columns: {list(kcs_df.columns)}")

# Load file LF-Log
lf_df = pd.read_csv('LF-Log.csv')
print(f"\nLF-Log data shape: {lf_df.shape}")
print(f"LF-Log columns: {list(lf_df.columns)}")

KCS data shape: (4394, 52)
KCS columns: ['ShiftName', 'ProductionDate', 'ProductGrade', 'GradeCode', 'ProductSizeCode', 'Length', 'BilletLotCode', 'BilletSampleName', 'OvenID', 'TotalWeight', 'C', 'Si', 'Mn', 'S', 'P', 'Cu', 'Ni', 'Cr', 'Mo', 'V', 'Ti', 'Al', 'Ca', 'B', 'Nb', 'CEV', 'O', 'N', 'H', 'Al_sol', 'Al_ins', 'Sn', 'As', 'Zr', 'Bi', 'Ce', 'Se', 'Te', 'Ta', 'Zn', 'La', 'TotalRange', 'Co60', 'Cs137', 'Ir192', 'Am241', 'pb', 'Cp', 'ClassifyID', 'ClassifyName', 'InputTime', 'OvenCode']

LF-Log data shape: (1728, 58)
LF-Log columns: ['ngay', 'Ca', 'me_tinh_luyen_so', 'mac_thep_yeu_cau', 'thoi_gian_vao_tinh_luyen', 'bat_dau', 'ket_thuc', 'thoi_gian_len_duc', 'thung_lf', 'lan_luyen_thu', 'nhiet_do_vao_tl', 'C', 'Si', 'Mn', 'S', 'P', 'khoi_luong_thung_thep', 'FeSi', 'FeMn', 'SiMn', 'than', 'FeCr', 'FeV', 'Niken', 'FeP', 'Cu', 'khac', 'huynh_thach', 'nhom_thoi', 'voi_song', 'dolomite', 'quaczit', 'day_feca', 'day_casi', 'day_ca_dac', 'xi_bao_on', 'thoi_gian_danh_dien', 'tieu_thu', 'C.1'

In [3]:
# Xem mẫu dữ liệu KCS
print("KCS sample (BilletLotCode column):")
print(kcs_df['BilletLotCode'].head(20).tolist())

KCS sample (BilletLotCode column):
['25A006006', '25C006270', '25B006096', '25C006271', '25B006097', '25C006272', '25C006273', '25B006098', '25B006099', '25C006274', '25B006100', '25A006009', '25D006564', '25B006101', '25D006565', '25B006102', '25D006566', '25B006103', '25D006567', '25B006105']


In [4]:
# Xem mẫu dữ liệu LF-Log
print("LF-Log sample (me_tinh_luyen_so column):")
print(lf_df['me_tinh_luyen_so'].head(20).tolist())

LF-Log sample (me_tinh_luyen_so column):
['D7695', 'C7518', 'A7129', 'A7130', 'A7132', 'A7133', 'C7524', 'C7525', 'C7526', 'C7527', 'C7528', 'C7529', 'C7530', 'C7531', 'A7142', 'C7533', 'C7534', 'C7535', 'C7536', 'C7537']


## 2. Phân Tích ID Mẻ

### Cấu trúc ID mẻ:
- **KCS (BilletLotCode)**: `25B006105` = 25 (năm) + B (lò) + 006105 (số mẻ 6 chữ số)
- **LF-Log (me_tinh_luyen_so)**: `C7536` = C (lò) + 7536 (số mẻ 4-5 chữ số)

### Logic để map:
- Từ KCS `25B006105` → trích xuất lò `B` và số mẻ `6105` (bỏ padding 0)
- Từ LF-Log `C7536` → trích xuất lò `C` và số mẻ `7536`
- Tạo khóa chung: `{lò}{số_mẻ}` (ví dụ: `B6105`, `C7536`)

In [5]:
def parse_kcs_heat_id(heat_id):
    """
    Parse KCS heat ID: 25B006105
    - 25 = year (2025)
    - B = furnace
    - 006105 = heat number (6 digits with leading zeros)
    
    Returns: (furnace, heat_number) e.g., ('B', 6105)
    """
    if pd.isna(heat_id) or not isinstance(heat_id, str):
        return None, None
    
    heat_id = str(heat_id).strip()
    
    # Kiểm tra format: 2 digit năm + 1 chữ cái lò + 6 digit số mẻ
    if len(heat_id) >= 9 and heat_id[:2].isdigit():
        furnace = heat_id[2]  # Lấy ký tự lò (A, B, C, D, E)
        heat_num_str = heat_id[3:]  # Phần còn lại là số mẻ
        try:
            heat_number = int(heat_num_str)  # Bỏ leading zeros
            return furnace.upper(), heat_number
        except ValueError:
            return None, None
    
    return None, None

def parse_lf_heat_id(heat_id):
    """
    Parse LF-Log heat ID: C7536
    - C = furnace
    - 7536 = heat number
    
    Returns: (furnace, heat_number) e.g., ('C', 7536)
    """
    if pd.isna(heat_id) or not isinstance(heat_id, str):
        return None, None
    
    heat_id = str(heat_id).strip()
    
    if len(heat_id) >= 2:
        furnace = heat_id[0]  # Ký tự đầu là lò
        heat_num_str = heat_id[1:]  # Phần còn lại là số mẻ
        try:
            heat_number = int(heat_num_str)
            return furnace.upper(), heat_number
        except ValueError:
            return None, None
    
    return None, None

# Test parsing functions
print("Test KCS parsing:")
test_kcs = ['25B006105', '25C007536', '25A006006', '25D006571']
for t in test_kcs:
    print(f"  {t} -> {parse_kcs_heat_id(t)}")

print("\nTest LF-Log parsing:")
test_lf = ['C7536', 'B7370', 'A7129', 'D7695']
for t in test_lf:
    print(f"  {t} -> {parse_lf_heat_id(t)}")

Test KCS parsing:
  25B006105 -> ('B', 6105)
  25C007536 -> ('C', 7536)
  25A006006 -> ('A', 6006)
  25D006571 -> ('D', 6571)

Test LF-Log parsing:
  C7536 -> ('C', 7536)
  B7370 -> ('B', 7370)
  A7129 -> ('A', 7129)
  D7695 -> ('D', 7695)


## 3. Tạo Khóa Chung Để Join

In [6]:
# Xử lý KCS - tạo khóa join
def create_kcs_join_key(row):
    furnace, heat_num = parse_kcs_heat_id(row['BilletLotCode'])
    if furnace and heat_num:
        return f"{furnace}{heat_num}"
    return None

kcs_df['heat_key'] = kcs_df.apply(create_kcs_join_key, axis=1)
kcs_df['furnace'] = kcs_df['BilletLotCode'].apply(lambda x: parse_kcs_heat_id(x)[0])
kcs_df['heat_number'] = kcs_df['BilletLotCode'].apply(lambda x: parse_kcs_heat_id(x)[1])

print("KCS heat_key samples:")
print(kcs_df[['BilletLotCode', 'furnace', 'heat_number', 'heat_key']].head(20))

KCS heat_key samples:
   BilletLotCode furnace  heat_number heat_key
0      25A006006       A         6006    A6006
1      25C006270       C         6270    C6270
2      25B006096       B         6096    B6096
3      25C006271       C         6271    C6271
4      25B006097       B         6097    B6097
5      25C006272       C         6272    C6272
6      25C006273       C         6273    C6273
7      25B006098       B         6098    B6098
8      25B006099       B         6099    B6099
9      25C006274       C         6274    C6274
10     25B006100       B         6100    B6100
11     25A006009       A         6009    A6009
12     25D006564       D         6564    D6564
13     25B006101       B         6101    B6101
14     25D006565       D         6565    D6565
15     25B006102       B         6102    B6102
16     25D006566       D         6566    D6566
17     25B006103       B         6103    B6103
18     25D006567       D         6567    D6567
19     25B006105       B         6105 

In [7]:
# Xử lý LF-Log - tạo khóa join
def create_lf_join_key(row):
    furnace, heat_num = parse_lf_heat_id(row['me_tinh_luyen_so'])
    if furnace and heat_num:
        return f"{furnace}{heat_num}"
    return None

lf_df['heat_key'] = lf_df.apply(create_lf_join_key, axis=1)
lf_df['lf_furnace'] = lf_df['me_tinh_luyen_so'].apply(lambda x: parse_lf_heat_id(x)[0])
lf_df['lf_heat_number'] = lf_df['me_tinh_luyen_so'].apply(lambda x: parse_lf_heat_id(x)[1])

print("LF-Log heat_key samples:")
print(lf_df[['me_tinh_luyen_so', 'lf_furnace', 'lf_heat_number', 'heat_key']].head(20))

LF-Log heat_key samples:
   me_tinh_luyen_so lf_furnace  lf_heat_number heat_key
0             D7695          D            7695    D7695
1             C7518          C            7518    C7518
2             A7129          A            7129    A7129
3             A7130          A            7130    A7130
4             A7132          A            7132    A7132
5             A7133          A            7133    A7133
6             C7524          C            7524    C7524
7             C7525          C            7525    C7525
8             C7526          C            7526    C7526
9             C7527          C            7527    C7527
10            C7528          C            7528    C7528
11            C7529          C            7529    C7529
12            C7530          C            7530    C7530
13            C7531          C            7531    C7531
14            A7142          A            7142    A7142
15            C7533          C            7533    C7533
16            C7534    

In [8]:
# Kiểm tra số lượng unique heat_key
print(f"KCS unique heat_keys: {kcs_df['heat_key'].nunique()}")
print(f"LF-Log unique heat_keys: {lf_df['heat_key'].nunique()}")

# Tìm các heat_key overlap
kcs_keys = set(kcs_df['heat_key'].dropna())
lf_keys = set(lf_df['heat_key'].dropna())
overlap_keys = kcs_keys.intersection(lf_keys)

print(f"\nOverlapping heat_keys: {len(overlap_keys)}")
print(f"Sample overlapping keys: {list(overlap_keys)[:20]}")

KCS unique heat_keys: 4359
LF-Log unique heat_keys: 1721

Overlapping heat_keys: 1057
Sample overlapping keys: ['C7550', 'C7721', 'C8096', 'C7919', 'A7138', 'A7155', 'D8079', 'B7446', 'C7973', 'B7563', 'C7760', 'A7289', 'B7368', 'D8409', 'B7626', 'C7716', 'D8493', 'C7743', 'C7940', 'D7798']


## 4. Merge Dữ Liệu

In [9]:
# Add prefix to LF columns to distinguish from KCS columns
lf_df_renamed = lf_df.add_prefix('lf_')
# Rename heat_key back for merging
lf_df_renamed = lf_df_renamed.rename(columns={'lf_heat_key': 'heat_key'})

# Merge on heat_key
merged_df = pd.merge(
    kcs_df,
    lf_df_renamed,
    on='heat_key',
    how='inner'  # Only keep matching records
)

print(f"Merged dataframe shape: {merged_df.shape}")
print(f"Number of merged records: {len(merged_df)}")

Merged dataframe shape: (1096, 115)
Number of merged records: 1096


In [10]:
# Xem các cột sau khi merge
print("Columns in merged dataframe:")
for i, col in enumerate(merged_df.columns):
    print(f"{i+1}. {col}")

Columns in merged dataframe:
1. ShiftName
2. ProductionDate
3. ProductGrade
4. GradeCode
5. ProductSizeCode
6. Length
7. BilletLotCode
8. BilletSampleName
9. OvenID
10. TotalWeight
11. C
12. Si
13. Mn
14. S
15. P
16. Cu
17. Ni
18. Cr
19. Mo
20. V
21. Ti
22. Al
23. Ca
24. B
25. Nb
26. CEV
27. O
28. N
29. H
30. Al_sol
31. Al_ins
32. Sn
33. As
34. Zr
35. Bi
36. Ce
37. Se
38. Te
39. Ta
40. Zn
41. La
42. TotalRange
43. Co60
44. Cs137
45. Ir192
46. Am241
47. pb
48. Cp
49. ClassifyID
50. ClassifyName
51. InputTime
52. OvenCode
53. heat_key
54. furnace
55. heat_number
56. lf_ngay
57. lf_Ca
58. lf_me_tinh_luyen_so
59. lf_mac_thep_yeu_cau
60. lf_thoi_gian_vao_tinh_luyen
61. lf_bat_dau
62. lf_ket_thuc
63. lf_thoi_gian_len_duc
64. lf_thung_lf
65. lf_lan_luyen_thu
66. lf_nhiet_do_vao_tl
67. lf_C
68. lf_Si
69. lf_Mn
70. lf_S
71. lf_P
72. lf_khoi_luong_thung_thep
73. lf_FeSi
74. lf_FeMn
75. lf_SiMn
76. lf_than
77. lf_FeCr
78. lf_FeV
79. lf_Niken
80. lf_FeP
81. lf_Cu
82. lf_khac
83. lf_huynh_thach
84.

## 5. Phân Tích Các Cột Trùng Lặp

In [11]:
# Phân tích các cột có thể trùng nội dung (thành phần hóa học)
# KCS columns for chemical composition
kcs_chem_cols = ['C', 'Si', 'Mn', 'S', 'P', 'Cu', 'Ni', 'Cr', 'Mo', 'V', 'Ti', 'Al', 'Ca', 'B', 'Nb']

# LF-Log columns for chemical composition (input and output)
# Input (before refining): C, Si, Mn, S, P
# Output (after refining): C.1, Si.1, Mn.1, S.1, P.1, Al, Ca.1

print("=== PHÂN TÍCH CÁC CỘT TRÙNG LẶP ===")
print("\n1. Thành phần hóa học trong KCS (kết quả lấy mẫu):")
print(f"   {kcs_chem_cols}")

print("\n2. Thành phần hóa học trong LF-Log:")
print("   - Đầu vào (trước tinh luyện): lf_C, lf_Si, lf_Mn, lf_S, lf_P")
print("   - Đầu ra (sau tinh luyện): lf_C.1, lf_Si.1, lf_Mn.1, lf_S.1, lf_P.1, lf_Al, lf_Ca.1")

print("\n3. CÁC CỘT BỊ TRÙNG NỘI DUNG:")
duplicate_analysis = [
    ('C (KCS)', 'lf_C.1 (LF output)', 'Carbon - thành phần carbon sau tinh luyện'),
    ('Si (KCS)', 'lf_Si.1 (LF output)', 'Silicon - thành phần silicon sau tinh luyện'),
    ('Mn (KCS)', 'lf_Mn.1 (LF output)', 'Manganese - thành phần mangan sau tinh luyện'),
    ('S (KCS)', 'lf_S.1 (LF output)', 'Sulfur - thành phần lưu huỳnh sau tinh luyện'),
    ('P (KCS)', 'lf_P.1 (LF output)', 'Phosphorus - thành phần photpho sau tinh luyện'),
    ('Al (KCS)', 'lf_Al (LF output)', 'Aluminum - thành phần nhôm'),
    ('Ca (KCS)', 'lf_Ca.1 (LF output)', 'Calcium - thành phần canxi'),
]

for kcs_col, lf_col, description in duplicate_analysis:
    print(f"   - {kcs_col} vs {lf_col}: {description}")

=== PHÂN TÍCH CÁC CỘT TRÙNG LẶP ===

1. Thành phần hóa học trong KCS (kết quả lấy mẫu):
   ['C', 'Si', 'Mn', 'S', 'P', 'Cu', 'Ni', 'Cr', 'Mo', 'V', 'Ti', 'Al', 'Ca', 'B', 'Nb']

2. Thành phần hóa học trong LF-Log:
   - Đầu vào (trước tinh luyện): lf_C, lf_Si, lf_Mn, lf_S, lf_P
   - Đầu ra (sau tinh luyện): lf_C.1, lf_Si.1, lf_Mn.1, lf_S.1, lf_P.1, lf_Al, lf_Ca.1

3. CÁC CỘT BỊ TRÙNG NỘI DUNG:
   - C (KCS) vs lf_C.1 (LF output): Carbon - thành phần carbon sau tinh luyện
   - Si (KCS) vs lf_Si.1 (LF output): Silicon - thành phần silicon sau tinh luyện
   - Mn (KCS) vs lf_Mn.1 (LF output): Manganese - thành phần mangan sau tinh luyện
   - S (KCS) vs lf_S.1 (LF output): Sulfur - thành phần lưu huỳnh sau tinh luyện
   - P (KCS) vs lf_P.1 (LF output): Phosphorus - thành phần photpho sau tinh luyện
   - Al (KCS) vs lf_Al (LF output): Aluminum - thành phần nhôm
   - Ca (KCS) vs lf_Ca.1 (LF output): Calcium - thành phần canxi


In [12]:
# So sánh giá trị thực tế giữa các cột trùng
comparison_pairs = [
    ('C', 'lf_C.1'),
    ('Si', 'lf_Si.1'),
    ('Mn', 'lf_Mn.1'),
    ('S', 'lf_S.1'),
    ('P', 'lf_P.1'),
    ('Al', 'lf_Al'),
]

print("=== SO SÁNH GIÁ TRỊ GIỮA CÁC CỘT TRÙNG ===")
for kcs_col, lf_col in comparison_pairs:
    if kcs_col in merged_df.columns and lf_col in merged_df.columns:
        kcs_vals = pd.to_numeric(merged_df[kcs_col], errors='coerce')
        lf_vals = pd.to_numeric(merged_df[lf_col], errors='coerce')
        
        diff = (kcs_vals - lf_vals).abs()
        print(f"\n{kcs_col} vs {lf_col}:")
        print(f"   Mean KCS: {kcs_vals.mean():.4f}, Mean LF: {lf_vals.mean():.4f}")
        print(f"   Mean absolute diff: {diff.mean():.4f}")
        print(f"   Correlation: {kcs_vals.corr(lf_vals):.4f}")

=== SO SÁNH GIÁ TRỊ GIỮA CÁC CỘT TRÙNG ===

C vs lf_C.1:
   Mean KCS: 0.0378, Mean LF: 0.0365
   Mean absolute diff: 0.0036
   Correlation: 0.3032

Si vs lf_Si.1:
   Mean KCS: 0.0123, Mean LF: 0.0125
   Mean absolute diff: 0.0005
   Correlation: 0.4343

Mn vs lf_Mn.1:
   Mean KCS: 0.1609, Mean LF: 0.7541
   Mean absolute diff: 0.5995
   Correlation: -0.0084

S vs lf_S.1:
   Mean KCS: 0.0034, Mean LF: 0.0035
   Mean absolute diff: 0.0001
   Correlation: 0.5120

P vs lf_P.1:
   Mean KCS: 0.0111, Mean LF: 0.0114
   Mean absolute diff: 0.0004
   Correlation: 0.4536

Al vs lf_Al:
   Mean KCS: 277.0346, Mean LF: 275.3922
   Mean absolute diff: 2.5526
   Correlation: 0.7652


## 6. Lưu Dữ Liệu Merged

In [13]:
# Lưu file merged
output_path = '01-data/merged_kcs_lf_data.csv'
merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"Saved merged data to: {output_path}")
print(f"Total rows: {len(merged_df)}")
print(f"Total columns: {len(merged_df.columns)}")

Saved merged data to: 01-data/merged_kcs_lf_data.csv
Total rows: 1096
Total columns: 115


## 7. Báo Cáo Chi Tiết Các Cột

In [14]:
# Tạo báo cáo chi tiết về các cột
column_descriptions = {
    # KCS columns
    'ShiftName': 'Ca sản xuất (ví dụ: 1A, 2B)',
    'ProductionDate': 'Ngày sản xuất',
    'ProductGrade': 'Loại sản phẩm (CT3, G60, SAE1006-Al)',
    'GradeCode': 'Mã loại thép',
    'ProductSizeCode': 'Kích thước sản phẩm (ví dụ: 85x1250)',
    'Length': 'Chiều dài phôi',
    'BilletLotCode': 'Mã lô phôi/ID mẻ (25B006105)',
    'BilletSampleName': 'Tên mẫu phôi (TSC1, TSC2)',
    'OvenID': 'ID lò',
    'TotalWeight': 'Tổng trọng lượng (kg)',
    'C': 'Carbon (%) - KCS',
    'Si': 'Silicon (%) - KCS',
    'Mn': 'Manganese (%) - KCS',
    'S': 'Sulfur (%) - KCS',
    'P': 'Phosphorus (%) - KCS',
    'Cu': 'Copper (%) - KCS',
    'Ni': 'Nickel (%) - KCS',
    'Cr': 'Chromium (%) - KCS',
    'Mo': 'Molybdenum (%) - KCS',
    'V': 'Vanadium (%) - KCS',
    'Ti': 'Titanium (ppm) - KCS',
    'Al': 'Aluminum (ppm) - KCS',
    'Ca': 'Calcium (ppm) - KCS',
    'B': 'Boron (ppm) - KCS',
    'Nb': 'Niobium (ppm) - KCS',
    'CEV': 'Carbon Equivalent Value',
    'O': 'Oxygen (ppm)',
    'N': 'Nitrogen (ppm)',
    'H': 'Hydrogen (ppm)',
    'Al_sol': 'Aluminum hòa tan (ppm)',
    'Al_ins': 'Aluminum không hòa tan (ppm)',
    'Sn': 'Tin (ppm)',
    'As': 'Arsenic (ppm)',
    'Zr': 'Zirconium (ppm)',
    'Bi': 'Bismuth (ppm)',
    'Ce': 'Cerium (ppm)',
    'Se': 'Selenium (ppm)',
    'Te': 'Tellurium (ppm)',
    'Ta': 'Tantalum (ppm)',
    'Zn': 'Zinc (ppm)',
    'La': 'Lanthanum (ppm)',
    'TotalRange': 'Phạm vi tổng',
    'Co60': 'Cobalt-60 (phóng xạ)',
    'Cs137': 'Cesium-137 (phóng xạ)',
    'Ir192': 'Iridium-192 (phóng xạ)',
    'Am241': 'Americium-241 (phóng xạ)',
    'pb': 'Lead (ppm)',
    'Cp': 'Cp index',
    'ClassifyID': 'ID phân loại',
    'ClassifyName': 'Tên phân loại (Loại 1, Loại 2)',
    'InputTime': 'Thời gian nhập liệu',
    'OvenCode': 'Mã lò (A, B, C, D, E)',
    'heat_key': 'Khóa join (tạo từ furnace + heat_number)',
    'furnace': 'Lò (từ BilletLotCode)',
    'heat_number': 'Số mẻ (từ BilletLotCode)',
    
    # LF-Log columns (with lf_ prefix)
    'lf_ngay': 'Ngày vận hành LF',
    'lf_Ca': 'Ca làm việc LF',
    'lf_me_tinh_luyen_so': 'ID mẻ tinh luyện (C7536)',
    'lf_mac_thep_yeu_cau': 'Mác thép yêu cầu',
    'lf_thoi_gian_vao_tinh_luyen': 'Thời gian vào tinh luyện',
    'lf_bat_dau': 'Thời gian bắt đầu',
    'lf_ket_thuc': 'Thời gian kết thúc',
    'lf_thoi_gian_len_duc': 'Thời gian lên đúc',
    'lf_thung_lf': 'Thùng LF',
    'lf_lan_luyen_thu': 'Lần luyện thứ',
    'lf_nhiet_do_vao_tl': 'Nhiệt độ vào tinh luyện (°C)',
    'lf_C': 'Carbon đầu vào (%) - LF',
    'lf_Si': 'Silicon đầu vào (%) - LF',
    'lf_Mn': 'Manganese đầu vào (%) - LF',
    'lf_S': 'Sulfur đầu vào (%) - LF',
    'lf_P': 'Phosphorus đầu vào (%) - LF',
    'lf_khoi_luong_thung_thep': 'Khối lượng thùng thép (kg)',
    'lf_FeSi': 'FeSi bổ sung (kg)',
    'lf_FeMn': 'FeMn bổ sung (kg)',
    'lf_SiMn': 'SiMn bổ sung (kg)',
    'lf_than': 'Than bổ sung (kg)',
    'lf_FeCr': 'FeCr bổ sung (kg)',
    'lf_FeV': 'FeV bổ sung (kg)',
    'lf_Niken': 'Niken bổ sung (kg)',
    'lf_FeP': 'FeP bổ sung (kg)',
    'lf_Cu': 'Cu bổ sung (kg)',
    'lf_khac': 'Nguyên liệu khác (kg)',
    'lf_huynh_thach': 'Huỳnh thạch (kg)',
    'lf_nhom_thoi': 'Nhôm thỏi (kg)',
    'lf_voi_song': 'Vite sống (kg)',
    'lf_dolomite': 'Dolomite (kg)',
    'lf_quaczit': 'Quaczit (kg)',
    'lf_day_feca': 'Dây FeCa (m)',
    'lf_day_casi': 'Dây CaSi (m)',
    'lf_day_ca_dac': 'Dây Ca đặc (m)',
    'lf_xi_bao_on': 'Xỉ bảo ôn (kg)',
    'lf_thoi_gian_danh_dien': 'Thời gian đánh điện (phút)',
    'lf_tieu_thu': 'Tiêu thụ điện',
    'lf_C.1': 'Carbon đầu ra (%) - LF [TRÙNG VỚI C của KCS]',
    'lf_Si.1': 'Silicon đầu ra (%) - LF [TRÙNG VỚI Si của KCS]',
    'lf_Mn.1': 'Manganese đầu ra (%) - LF [TRÙNG VỚI Mn của KCS]',
    'lf_S.1': 'Sulfur đầu ra (%) - LF [TRÙNG VỚI S của KCS]',
    'lf_P.1': 'Phosphorus đầu ra (%) - LF [TRÙNG VỚI P của KCS]',
    'lf_Al': 'Aluminum đầu ra (ppm) - LF [TRÙNG VỚI Al của KCS]',
    'lf_Ca.1': 'Calcium đầu ra (ppm) - LF [TRÙNG VỚI Ca của KCS]',
    'lf_lan_1': 'Lần lấy mẫu 1',
    'lf_ra_thep': 'Nhiệt độ ra thép (°C)',
    'lf_nhiet_do_duc_yeu_cau': 'Nhiệt độ đúc yêu cầu (°C)',
    'lf_nhiet_do_do_tren_duc': 'Nhiệt độ đo trên đúc (°C)',
    'lf_thoi_gian_dinh_tre': 'Thời gian đình trệ (phút)',
    'lf_ly_do_dinh_tre': 'Lý do đình trệ',
    'lf_ghi_chu_1': 'Ghi chú 1',
    'lf_thoi_gian_bat_dau_thoi_mem': 'Thời gian bắt đầu thổi mềm',
    'lf_thoi_gian_ket_thu_thoi_mem': 'Thời gian kết thúc thổi mềm',
    'lf_tong_thoi_gian_thoi_mem': 'Tổng thời gian thổi mềm',
    'lf_tinh_trang_xi_lo_thoi_qua_tinh_luyen': 'Tình trạng xỉ lò thổi qua tinh luyện',
    'lf_tinh_trang_xi': 'Tình trạng xỉ',
    'lf_ghi_chu': 'Ghi chú',
}

# In báo cáo
print("=" * 100)
print("BÁO CÁO CHI TIẾT CÁC CỘT SAU KHI MERGE")
print("=" * 100)

for col in merged_df.columns:
    desc = column_descriptions.get(col, 'Không có mô tả')
    print(f"\n{col}:")
    print(f"   Mô tả: {desc}")
    print(f"   Dtype: {merged_df[col].dtype}")
    print(f"   Non-null: {merged_df[col].notna().sum()}/{len(merged_df)}")

BÁO CÁO CHI TIẾT CÁC CỘT SAU KHI MERGE

ShiftName:
   Mô tả: Ca sản xuất (ví dụ: 1A, 2B)
   Dtype: object
   Non-null: 1096/1096

ProductionDate:
   Mô tả: Ngày sản xuất
   Dtype: object
   Non-null: 1096/1096

ProductGrade:
   Mô tả: Loại sản phẩm (CT3, G60, SAE1006-Al)
   Dtype: object
   Non-null: 1096/1096

GradeCode:
   Mô tả: Mã loại thép
   Dtype: object
   Non-null: 1096/1096

ProductSizeCode:
   Mô tả: Kích thước sản phẩm (ví dụ: 85x1250)
   Dtype: object
   Non-null: 1096/1096

Length:
   Mô tả: Chiều dài phôi
   Dtype: float64
   Non-null: 1096/1096

BilletLotCode:
   Mô tả: Mã lô phôi/ID mẻ (25B006105)
   Dtype: object
   Non-null: 1096/1096

BilletSampleName:
   Mô tả: Tên mẫu phôi (TSC1, TSC2)
   Dtype: object
   Non-null: 1096/1096

OvenID:
   Mô tả: ID lò
   Dtype: int64
   Non-null: 1096/1096

TotalWeight:
   Mô tả: Tổng trọng lượng (kg)
   Dtype: float64
   Non-null: 1096/1096

C:
   Mô tả: Carbon (%) - KCS
   Dtype: float64
   Non-null: 1096/1096

Si:
   Mô tả: Silic

In [15]:
# Tóm tắt các cột trùng lặp
print("\n" + "=" * 100)
print("TÓM TẮT CÁC CỘT BỊ TRÙNG LẶP")
print("=" * 100)

duplicate_cols = [
    ('C', 'lf_C.1', 'Carbon sau tinh luyện'),
    ('Si', 'lf_Si.1', 'Silicon sau tinh luyện'),
    ('Mn', 'lf_Mn.1', 'Manganese sau tinh luyện'),
    ('S', 'lf_S.1', 'Sulfur sau tinh luyện'),
    ('P', 'lf_P.1', 'Phosphorus sau tinh luyện'),
    ('Al', 'lf_Al', 'Aluminum sau tinh luyện'),
    ('Ca', 'lf_Ca.1', 'Calcium sau tinh luyện'),
]

print("\nLƯU Ý: Các cột sau có thể chứa dữ liệu tương tự (cùng đo thành phần hóa học sau tinh luyện):")
print("-" * 80)
print(f"{'KCS Column':<15} {'LF-Log Column':<20} {'Ý nghĩa':<45}")
print("-" * 80)
for kcs_col, lf_col, meaning in duplicate_cols:
    print(f"{kcs_col:<15} {lf_col:<20} {meaning:<45}")

print("\nGIẢI THÍCH:")
print("- KCS: Dữ liệu từ phòng Kiểm soát Chất lượng (lấy mẫu tại đúc)")
print("- LF (đầu vào: lf_C, lf_Si...): Thành phần trước khi tinh luyện")
print("- LF (đầu ra: lf_C.1, lf_Si.1...): Thành phần sau khi tinh luyện")
print("\n=> KCS thường được lấy mẫu sau đúc, nên có thể khác biệt nhỏ so với lf_*.1")


TÓM TẮT CÁC CỘT BỊ TRÙNG LẶP

LƯU Ý: Các cột sau có thể chứa dữ liệu tương tự (cùng đo thành phần hóa học sau tinh luyện):
--------------------------------------------------------------------------------
KCS Column      LF-Log Column        Ý nghĩa                                      
--------------------------------------------------------------------------------
C               lf_C.1               Carbon sau tinh luyện                        
Si              lf_Si.1              Silicon sau tinh luyện                       
Mn              lf_Mn.1              Manganese sau tinh luyện                     
S               lf_S.1               Sulfur sau tinh luyện                        
P               lf_P.1               Phosphorus sau tinh luyện                    
Al              lf_Al                Aluminum sau tinh luyện                      
Ca              lf_Ca.1              Calcium sau tinh luyện                       

GIẢI THÍCH:
- KCS: Dữ liệu từ phòng Kiểm soát Chấ

In [16]:
# Hiển thị thống kê cuối cùng
print("\n" + "=" * 100)
print("THỐNG KÊ CUỐI CÙNG")
print("=" * 100)
print(f"\nTổng số bản ghi KCS ban đầu: {len(kcs_df)}")
print(f"Tổng số bản ghi LF-Log ban đầu: {len(lf_df)}")
print(f"Tổng số bản ghi sau khi merge (inner join): {len(merged_df)}")
print(f"Tổng số cột sau khi merge: {len(merged_df.columns)}")
print(f"\nFile đã lưu: {output_path}")


THỐNG KÊ CUỐI CÙNG

Tổng số bản ghi KCS ban đầu: 4394
Tổng số bản ghi LF-Log ban đầu: 1728
Tổng số bản ghi sau khi merge (inner join): 1096
Tổng số cột sau khi merge: 115

File đã lưu: 01-data/merged_kcs_lf_data.csv
